# Set your API Endpoint

In [ ]:
import autogen
from pathlib import Path

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-0125-preview", "gpt-3.5-turbo"],
    },
)

# Construct Agents

In [ ]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Client",
    system_message="A human client. Interact with the CEO to discuss your Crypto report needs.",
    code_execution_config=False,
)

ABS_PATH: Path = Path(__file__).parent.absolute()
agency_system_message = ABS_PATH.joinpath("instructions/agency_system_message.md").open("r").read()
ceo_system_message = ABS_PATH.joinpath("instructions/agents/systems/ceo.md").open("r").read()
marketAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/marketAnalyst.md").open("r").read()
fundamentalResearchAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/fundamentalResearchAnalyst.md").open("r").read()
investmentResearchAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/investmentResearchAnalyst.md").open("r").read()
riskManagementSpecialist_system_message = ABS_PATH.joinpath("instructions/agents/systems/riskManagementSpecialist.md").open("r").read()
socialMediaAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/socialMediaAnalyst.md").open("r").read()

ceo = autogen.AssistantAgent(
    name="CEO",
    llm_config=gpt4_config,
    system_message=ceo_system_message + "\n\nUse the following as the corporate official guide:\n\n" + agency_system_message,
)

marketAnalyst = autogen.AssistantAgent(
    name="Market Analyst",
    llm_config=gpt4_config,
    system_message=marketAnalyst_system_message,
)

fundamentalResearchAnalyst = autogen.AssistantAgent(
    name="Fundamental Research Analyst",
    llm_config=gpt4_config,
    system_message=fundamentalResearchAnalyst_system_message,
)

investmentResearchAnalyst = autogen.AssistantAgent(
    name="Investment Research Analyst",
    llm_config=gpt4_config,
    system_message=investmentResearchAnalyst_system_message,
)

riskManagementSpecialist = autogen.AssistantAgent(
    name="Risk Management Specialist",
    llm_config=gpt4_config,
    system_message=riskManagementSpecialist_system_message,
)

socialMediaAnalyst = autogen.AssistantAgent(
    name="Social Media Analyst",
    llm_config=gpt4_config,
    system_message=socialMediaAnalyst_system_message,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, ceo, marketAnalyst, fundamentalResearchAnalyst, investmentResearchAnalyst, riskManagementSpecialist, socialMediaAnalyst], messages=[], max_round=50
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

# Start Chat

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
I want a general report on the crypto market for today.
""",
)